# Data wrangling and datetimes

Dates and times are a special kind of data type. In this notebook, we will keep exploring the `orders` and `orderlines` datasets from Eniac and learn to deal with them. 

In [1]:
import pandas as pd

In [2]:
url = 'https://drive.google.com/file/d/14Y7g5ITyf6LMyPoKc9wr010V9StaCUux/view?usp=sharing' # orderlines.csv
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
order_lines = pd.read_csv(path)

In [3]:
url = 'https://drive.google.com/file/d/1BLEHcP-9fm9Rv7A01H3co2XBMnSr66YC/view?usp=sharing' # orders.csv
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
orders = pd.read_csv(path)

## Data exploration

**Exercise**: check whether the dates from the `orderlines` dataset match with the dates on the `orders` dataset.

In [4]:
order_lines.head()

,id,id_order,product_id,product_quantity,sku,unit_price,date
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38


In [5]:
order_lines.shape

(293983, 7)

In [6]:
orders.head()

,order_id,created_date,total_paid,state
0,241319,2017-01-02 13:35:40,44.99,Cancelled
1,241423,2017-11-06 13:10:02,136.15,Completed
2,242832,2017-12-31 17:40:03,15.76,Completed
3,243330,2017-02-16 10:59:38,84.98,Completed
4,243784,2017-11-24 13:35:19,157.86,Cancelled


In [7]:
date_com = pd.merge(order_lines, orders.rename(columns={'order_id': 'id_order'}), how='left')
date_com.head()

,id,id_order,product_id,product_quantity,sku,unit_price,date,created_date,total_paid,state
0,1119109,299539,0,1,OTT0133,18.99,2017-01-01 00:07:19,2017-01-01 00:07:19,18.99,Shopping Basket
1,1119110,299540,0,1,LGE0043,399.00,2017-01-01 00:19:45,2017-01-01 00:19:45,399.00,Shopping Basket
2,1119111,299541,0,1,PAR0071,474.05,2017-01-01 00:20:57,2017-01-01 00:20:57,474.05,Shopping Basket
3,1119112,299542,0,1,WDT0315,68.39,2017-01-01 00:51:40,2017-01-01 00:51:40,68.39,Shopping Basket
4,1119113,299543,0,1,JBL0104,23.74,2017-01-01 01:06:38,2017-01-01 01:06:38,23.74,Shopping Basket


In [8]:
date_com.shape

(293983, 10)

In [9]:
type(date_com)

pandas.core.frame.DataFrame

In [10]:
date_com.dtypes

id                    int64
id_order              int64
product_id            int64
product_quantity      int64
sku                  object
unit_price           object
date                 object
created_date         object
total_paid          float64
state                object
dtype: object

In [11]:
date_com['date'] = pd.to_datetime(date_com['date'])

In [12]:
date_com['created_date'] = pd.to_datetime(date_com['created_date'])

In [13]:
date_com.dtypes

id                           int64
id_order                     int64
product_id                   int64
product_quantity             int64
sku                         object
unit_price                  object
date                datetime64[ns]
created_date        datetime64[ns]
total_paid                 float64
state                       object
dtype: object

In [14]:
# code here
date_com['com_datetimes'] = date_com['date'] == date_com['created_date']
date_com['com_datetimes'].head()

0    True
1    True
2    True
3    True
4    True
Name: com_datetimes, dtype: bool

In [15]:
date_com['com_datetimes'].value_counts()

True     166451
False    127532
Name: com_datetimes, dtype: int64

In [16]:
date_com.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 293983 entries, 0 to 293982
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   id                293983 non-null  int64         
 1   id_order          293983 non-null  int64         
 2   product_id        293983 non-null  int64         
 3   product_quantity  293983 non-null  int64         
 4   sku               293983 non-null  object        
 5   unit_price        293983 non-null  object        
 6   date              293983 non-null  datetime64[ns]
 7   created_date      293749 non-null  datetime64[ns]
 8   total_paid        293743 non-null  float64       
 9   state             293749 non-null  object        
 10  com_datetimes     293983 non-null  bool          
dtypes: bool(1), datetime64[ns](2), float64(1), int64(4), object(3)
memory usage: 25.0+ MB


In [17]:
date_com['date'].isna().value_counts()

False    293983
Name: date, dtype: int64

In [18]:
date_com['created_date'].isna().value_counts()

False    293749
True        234
Name: created_date, dtype: int64

In [19]:
import datetime

datetime.date.fromisoformat("1990-07-28")

datetime.date(1990, 7, 28)

# CHALLENGES

**1. Which are the top 6 orders with the highest number of products sold?**

In [20]:
(
date_com[date_com['state'] == 'Completed']
    .groupby('id_order')
    .product_quantity
    .sum()
    .sort_values(ascending=False)
    .head(6)
)

id_order
438871    72
495987    50
507768    37
300475    35
432242    35
468784    33
Name: product_quantity, dtype: int64

**2. Which are the top 6 orders for the 1 of July of 2017?**

Combining `.assign()` with `.dt.date` will help you extract the date of a datetime column and use this information to filter it. Use the `datetime` module and its `date.fromisoformat()` method as we showed above.

Here is an example how to filter the first of January 2017.

In [21]:
(
date_com[(date_com['date'].dt.date == datetime.date.fromisoformat('2017-07-01')) & (date_com['state'] == 'Completed')]
    .groupby('id_order') 
    .product_quantity
    .sum()
    .sort_values(ascending=False)
    .head(6)
)

id_order
371309    7
371263    4
370965    3
371052    3
371291    3
371293    3
Name: product_quantity, dtype: int64

**3. Which is the month with the highest number of units sold?** 

Remember to look at the documentation of [`datetime.strftime()`](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior) 

In [22]:
(
date_com.assign(month = date_com['date'].dt.strftime('%B')).assign(year = date_com['date'].dt.strftime('%Y')).copy()[date_com['state'] == 'Completed']
    .groupby(['year', 'month'])
    .agg(prdt_sum=('product_quantity','sum'))
    .sort_values(by='prdt_sum', ascending=False)
    .groupby('year')
    .head(1)
)

,,prdt_sum
year,month,
2017,November,11190
2018,January,7688


The results enormous differences between some months. Why do you think this is happening? Do we only have one year of data? In case you have multiple years, filter for only years that are complete to find out the best month for that year.

Tip: the output of the function `.dt.strftime()` is a string, so if you use it in a condition it will only match with strings!

**4. For each month, find the day of the week with the highest amount of products sold** 


In [23]:
(
date_com
    .assign(month = date_com['date'].dt.strftime('%B'))
    .assign(year = date_com['date'].dt.strftime('%Y'))
    .assign(week_day = date_com['date'].dt.strftime('%A'))
    .copy()[date_com['state'] == 'Completed']
    .groupby(['year', 'month', 'week_day'])
    .agg(prdt_sum=('product_quantity','sum'))
    .sort_values(by='prdt_sum', ascending=False)
    .groupby(['year', 'month'])
    .head(1)
)

prdt_sum
year month     week_day           
2017 November  Friday         3043
     January   Monday         2010
2018 January   Tuesday        1718
2017 December  Friday         1373
     July      Monday         1368
     October   Monday         1142
2018 February  Wednesday      1013
2017 February  Wednesday       709
     August    Tuesday         695
     September Monday          647
     May       Wednesday       617
2018 March     Friday          495
2017 June      Thursday        489
     April     Friday          483
     March     Friday           71

In [24]:
(
date_com
    .assign(month = date_com['date'].dt.strftime('%B'))
    .assign(year = date_com['date'].dt.strftime('%Y'))
    .assign(week_day = date_com['date'].dt.strftime('%A'))
    .copy()[date_com['state'] == 'Completed']
    .groupby(['year', 'month', 'week_day'])
    .agg(prdt_sum=('product_quantity','sum'))
    .reset_index()
    .sort_values(by='prdt_sum', ascending=False)
    .groupby(['year', 'month'])
    .head(1)
)

,year,month,week_day,prdt_sum
63,2017,November,Friday,3043
29,2017,January,Monday,2010
96,2018,January,Tuesday,1718
14,2017,December,Friday,1373
36,2017,July,Monday,1368
71,2017,October,Monday,1142
90,2018,February,Wednesday,1013
27,2017,February,Wednesday,709
12,2017,August,Tuesday,695
78,2017,September,Monday,647


In [25]:
(
date_com
    .assign(month = date_com['date'].dt.strftime('%B'))
    .assign(year = date_com['date'].dt.strftime('%Y'))
    .assign(week_day = date_com['date'].dt.strftime('%A'))
    .copy()[date_com['state'] == 'Completed']
    .groupby(['year', 'month', 'week_day'])
    .agg(prdt_sum=('product_quantity','sum'))
    .sort_values(by='prdt_sum', ascending=False)
    .reset_index()
    .groupby(['year', 'month'])
    .head(1)
    .sort_values(by=['year', 'month'], ascending=True)
)

,year,month,week_day,prdt_sum
59,2017,April,Friday,483
36,2017,August,Tuesday,695
8,2017,December,Friday,1373
34,2017,February,Wednesday,709
2,2017,January,Monday,2010
9,2017,July,Monday,1368
58,2017,June,Thursday,489
98,2017,March,Friday,71
45,2017,May,Wednesday,617
0,2017,November,Friday,3043
